In [1]:
from pathlib import Path
import sys, sqlite3, json, pandas as pd

# Текущая рабочая директория Jupyter
cwd = Path.cwd()

# Если CWD = Project/notebooks, то корень проекта = parent
project_root = str(cwd.parent)

# Добавляем корень проекта в sys.path (в начало, чтобы он имел приоритет)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

pd.set_option("display.width", 2000)     # максимальная ширина в символах
pd.set_option("display.max_colwidth", None)

from config import DB_PATH, PARQUETS_DIR, CTX_JSONLS_DIR
from mlapp import build_features_from_ctx, error_tokenizer

In [2]:
df = pd.read_parquet(PARQUETS_DIR / "data_marked_after_EOF.parquet")
ctxs_without_macros = pd.read_json(CTX_JSONLS_DIR / "data_without_macros.jsonl", lines=True)
ctxs_with_macros = pd.read_json(CTX_JSONLS_DIR / "data_with_macros.jsonl", lines=True)

In [3]:
ctxs = pd.concat([ctxs_without_macros, ctxs_with_macros], axis=0)
df = df.join(ctxs.set_index("row_id"), how="inner", validate="one_to_one")

In [4]:
def to_ctx_cols(ctx : dict) -> tuple[str | None, str | None]:
    if not isinstance(ctx, dict) or not ctx:
        return None, None
    
    tokens, numeric = build_features_from_ctx(ctx)
    return json.dumps(tokens, ensure_ascii=False), json.dumps(numeric, ensure_ascii=False)

In [5]:
tmp = df["ctx"].apply(to_ctx_cols)
df["ctx_tokens"] = tmp.apply(lambda x: x[0])
df["ctx_numeric"] = tmp.apply(lambda x: x[1])

In [6]:
def to_error_text_tokens(text : str) -> str:
    tokens = error_tokenizer(text)
    return json.dumps(tokens, ensure_ascii=False)

In [7]:
df["error_text_tokens"] = df["error_text"].apply(to_error_text_tokens)

In [8]:
cols = [
    "label_code",
    "error_text",
    "error_text_tokens",
    "error_code",
    "error_line",
    "source_code",
    "ctx_tokens",
    "ctx_numeric",
    "is_in_train",
]

df_out = df[cols].copy()

In [9]:
with sqlite3.connect(DB_PATH) as conn:
    df_out.to_sql("temp_table", conn, if_exists="replace", index=False,
        dtype={
            "label_code": "text",
            "error_text": "text",
            "error_text_tokens": "json",
            "error_code": "text",
            "error_line": "integer",
            "source_code": "text",
            "ctx_tokens": "json",
            "ctx_numeric": "json",
            "is_in_train": "integer"
        })

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute("""
                    WITH temp2 AS (
                        WITH temp1 AS (
                            SELECT tt.*, e.error_code_id
                            FROM temp_table tt
                            JOIN error_codes e on tt.error_code = e.error_code
                        )
                        SELECT temp1.*, r.recommendation_id
                        FROM temp1
                        JOIN recommendations r on temp1.label_code = r.recommendation_code AND temp1.error_code_id = r.error_code_id
                    )
                    INSERT INTO training_data (label, error_text, error_text_tokens, error_code, error_line, source_code, ctx_tokens, ctx_numeric, is_in_train)
                    SELECT recommendation_id, error_text, error_text_tokens, error_code, error_line, source_code, ctx_tokens, ctx_numeric, is_in_train
                    FROM temp2;
                """)
    cur.execute("""
                    DROP TABLE temp_table;
                """)